In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NLP_Example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 12:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 12:17:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.csv("SMSSpamCollection", inferSchema=True, sep="\t")

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [7]:
data = data.withColumnsRenamed(colsMap={"_c0": "class", "_c1": "text"})

In [8]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn("length", length(data["text"]))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
data.groupBy("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [13]:
from pyspark.ml.feature import (
    Tokenizer,
    StopWordsRemover,
    CountVectorizer,
    IDF,
    StringIndexer,
)

In [22]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stop_remove = StopWordsRemover(inputCol="token_text", outputCol="stop_token")
count_vec = CountVectorizer(inputCol="stop_token", outputCol="c_vec")
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_numeric = StringIndexer(inputCol="class", outputCol="label")

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
clean_up = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

In [17]:
from pyspark.ml.classification import NaiveBayes

In [18]:
nb = NaiveBayes()

In [19]:
from pyspark.ml import Pipeline

In [23]:
data_prep_pipe = Pipeline(
    stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up]
)

In [24]:
cleaner = data_prep_pipe.fit(data)

In [25]:
clean_data = cleaner.transform(data)

In [26]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [27]:
clean_data = clean_data.select("label", "features")

In [28]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [29]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [30]:
spam_detector = nb.fit(training)

24/03/16 12:48:41 WARN DAGScheduler: Broadcasting large task binary with size 1166.5 KiB
24/03/16 12:48:43 WARN DAGScheduler: Broadcasting large task binary with size 1146.0 KiB


In [31]:
test_results = spam_detector.transform(test)

In [32]:
test_results.show()

24/03/16 12:49:42 WARN DAGScheduler: Broadcasting large task binary with size 1371.5 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1058.2887175144...|[1.0,8.5089807784...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-693.27137467060...|[1.0,8.9294310560...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1294.2554113235...|[1.0,7.0742012638...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1140.7120145487...|[1.01152013998164...|       1.0|
|  0.0|(13424,[0,1,881,1...|[-98.302627244842...|[0.99999972263748...|       0.0|
|  0.0|(13424,[0,1,881,1...|[-99.986824831783...|[0.99999978738115...|       0.0|
|  0.0|(13424,[0,2,3,5,6...|[-2578.6186923836...|[1.0,9.7108564537...|       0.0|
|  0.0|(13424,[0,2,3,8,2...|[-1609.3837073356...|[1.0,1.1910111884...|       0.0|
|  0.0|(13424,[0,2,4,5,1...|[-2510.0810418964...|[1.0,9.9110951569...|       0.0|
|  0.0|(13424,[0

24/03/16 12:49:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/16 12:49:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()

In [35]:
acc = acc_eval.evaluate(test_results)

24/03/16 12:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1376.5 KiB


In [36]:
print("ACC of NB Model")
print(acc)

ACC of NB Model
0.9258437081190222
